In [1]:
from selenium import webdriver
from selenium.webdriver import ActionChains
from bs4 import BeautifulSoup as bs
import requests
import pandas as pd
import html5lib
import re
import time

In [2]:
#Scraping from website for prison data
base_url = "https://www.prisonstudies.org/world-prison-brief-data"
req = requests.get(base_url)
soup = bs(req.content)
results = soup.find_all('div', class_="views-field views-field-title")
country_url_list = [f'https://www.prisonstudies.org{country.a["href"]}' for country in results]


In [3]:
labels = []
for url in country_url_list[0:1]:
    req = requests.get(url)
    soup = bs(req.text,'lxml')
    tables = soup.find_all('table')
    table = tables[2].find_all('tr')
    for row in table:
        labels.append(row.th.text)

In [4]:
prison_df = pd.DataFrame(columns=labels)
for url in country_url_list:
    req = requests.get(url)
    soup = bs(req.text,'lxml')
    tables = soup.find_all('table')
    dict = {}

    country = tables[0].find('td',class_="field-content").text
    dict.setdefault('country', []).append(country)

    table = tables[2].find_all('tr')
    for row in table:
        label = row.th.text
        try:
            number = row.find(
                'div',class_='field field-name-field-integer field-type-number-integer field-label-hidden').\
                    find('div',class_="field-item even").text
            dict.setdefault(f'{label}',[]).append(number)
        except:
            pass
        try:
            percent = row.find(
            'div',class_='field field-name-field-percentage field-type-number-decimal field-label-hidden').\
                find('div',class_="field-item even").text
            dict.setdefault(f'{label}',[]).append(percent)
        except:
            pass
    df = pd.DataFrame.from_dict(dict)
    prison_df = prison_df.append(df)

In [5]:
#clean data in df
columns = ['prison_pop', 'prison_rate','pre_trial_detainees',
             'women_percent', 'minors_percent','foreigns_percent',
             'number_institution','capacity','occupancy','country']
final_prison_df = prison_df.copy()
for i,c in enumerate(columns[0:-1]):
    final_prison_df.rename({final_prison_df.columns[i]:c}, axis=1,inplace=True)
    final_prison_df[c] = final_prison_df[c].str.replace(" ","")
    final_prison_df[c] = final_prison_df[c].str.replace("%","")

In [6]:
patterns = ['[/]\w+','\[\w+$','\,\s+\w+\s?\w+\s?\w+',
'^\w+\s+\w+\:\s+','\:\s+\w+','\:\s+\w+\s+\w+']
for country in final_prison_df['country']:
    for pattern in patterns:
        try:
            new_c = re.sub(pattern,'', country)
            final_prison_df['country'] = final_prison_df['country'].\
                replace(country,new_c)
            final_prison_df['country'] = final_prison_df['country'].str.\
                replace("St\.",'Saint')
            final_prison_df['country'] = final_prison_df['country'].str.\
                replace("Democratic People[']s Republic of [(]North[)] Korea",\
                    'North Korea')
            final_prison_df['country'] = final_prison_df['country'].str.\
                replace('Republic of [(]South[)] Korea','South Korea')
            final_prison_df['country'] = final_prison_df['country'].str.\
                replace('Democratic Republic of Congo','Congo (DRC)')
            final_prison_df['country'] = final_prison_df['country'].str.\
                replace('Scotland','United Kingdom')
            final_prison_df['country'] = final_prison_df['country'].str.\
                replace('Russian Federation','Russia')
            final_prison_df['country'] = final_prison_df['country'].str.\
                replace('Faeroe','Faroe')
            final_prison_df['country'] = final_prison_df['country'].str.\
                replace('Virgin Islands (USA)','U.S Virgin Islands')
        except:
            continue


In [7]:
#final clean
final_prison_df = final_prison_df.fillna(0)
final_prison_df.iloc[:,[1,6,7]] = final_prison_df.iloc[:,[1,6,7]].astype('int')
final_prison_df.iloc[:,[2,3,4,5,8]] = final_prison_df.iloc[:,[2,3,4,5,8]].astype('float')
final_prison_df.iloc[:,9] = final_prison_df.iloc[:,[9]].astype('string')
final_prison_df = final_prison_df.iloc[:,[9,0,1,2,3,4,5,6,7,8]].\
    sort_values('country').reset_index(drop=True)
final_prison_df = final_prison_df.drop(26)
f = open('../Resources/prison_pop.csv','w', encoding='utf-8')
final_prison_df.to_csv(f,index=False)
f.close()

In [8]:
final_prison_df.head()

,country,prison_pop,prison_rate,pre_trial_detainees,women_percent,minors_percent,foreigns_percent,number_institution,capacity,occupancy
0,Afghanistan,30748,87,31.3,2.6,4,0.9,251,16711,179.9
1,Albania,4508,159,45.7,1.9,0.2,1.5,23,6236,85
2,Algeria,65000,153,12,1.5,0.8,3.8,131,68317,89.3
3,American Samoa (USA),193,345,14.9,5.5,0,62.2,1,142,166.2
4,Andorra,43,55,42.2,11.1,0,88.5,1,142,30.3
